In [1]:
from neuronx_distributed_inference.utils.testing import build_module

/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise.py:68: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  component, error = import_nki(config)
/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/

In [2]:
import torch


def piecewise_safe_division(a, b):
    """
    Piecewise safe division of tensors a/b with special rules for division by zero.
    Unravels/flattens `a` and `b` if not already 1-D (handled by summary_stats).
    """
    is_zero = b == 0

    # Compute x/y where y != 0
    div = a / b

    # For x==0 & y==0 --> 0
    case1 = (a == 0) & is_zero

    # For x<0 & y==0 --> -inf
    case2 = (a < 0) & is_zero

    # For x>0 & y==0 --> inf
    case3 = (a > 0) & is_zero

    # Start with normal division
    c = div

    # Set 0 where x==0 & y==0
    c = torch.where(case1, torch.zeros_like(c), c)

    # Set -inf where x<0 & y==0
    c = torch.where(case2, torch.full_like(c, -float('inf')), c)

    # Set inf where x>0 & y==0
    c = torch.where(case3, torch.full_like(c, float('inf')), c)

    return c


def summary_stats(A, B):
    if isinstance(A, torch.Tensor) and isinstance(B, torch.Tensor):
        with torch.no_grad():
            # 1. Check matching shape
            if A.shape != B.shape:
                raise ValueError(f"Tensor shape mismatch: {A.shape} vs {B.shape}")

            # 2. Flatten inputs
            A = A.contiguous().view(-1)
            B = B.contiguous().view(-1)

            diff = A - B

            abs_error = torch.abs(diff)
            min_abs_error = torch.min(abs_error)
            median_abs_error = torch.median(abs_error)
            max_abs_error = torch.max(abs_error)

            denominator = torch.abs(A) + torch.abs(B)
            rel_error = piecewise_safe_division(abs_error, denominator)
            min_rel_error = torch.min(rel_error)
            median_rel_error = torch.median(rel_error)
            max_rel_error = torch.max(rel_error)
            
            print("Tensor Comparison Results:\n")
            print(f"Absolute Error (min/median/max):  {min_abs_error:.4g}/{median_abs_error:.4g}/{max_abs_error:.4g}")
            print(f"Relative Error (min/median/max):  {min_rel_error:.4g}/{median_rel_error:.4g}/{max_rel_error:.4g}")
    elif isinstance(A, tuple) and isinstance(B, tuple):
        if len(A) != len(B):
            raise ValueError(f"Tuple length mismatch: {len(A)} vs {len(B)}")

        for i, (a, b) in enumerate(zip(A, B)):
            print(f"Element {i}:")
            print()

            summary_stats(a, b)
            print()
    else:
        raise ValueError(f"Unknown and/or mismatching types: {type(A)} and {type(B)}")
        

In [3]:
import json
import math

import torch
from torch import nn
from torch.nn import functional as F
from torch._dynamo._trace_wrapped_higher_order_op import TransformGetItemToIndex
from typing import Optional, Tuple


MAX_POSITION_EMBEDDINGS = 131072
MAX_LENGTH = 20
TOP_K = 50
EOS_TOKEN_ID = [200002, 199999]
PAD_TOKEN_ID = 199999


class GptOssTopKRouter(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.empty(32, 2880, dtype=torch.bfloat16))
        self.bias = nn.Parameter(torch.empty(32, dtype=torch.bfloat16))

    def forward(self, hidden_states):
        hidden_states = hidden_states.reshape(-1, 2880)
        router_logits = F.linear(hidden_states, self.weight, self.bias)  # (seq_len, num_experts)
        router_top_value, router_indices = torch.topk(router_logits, 4, dim=-1)  # (seq_len, top_k)
        router_top_value = torch.nn.functional.softmax(router_top_value, dim=1, dtype=router_top_value.dtype)
        router_scores = torch.zeros_like(router_logits).scatter_(1, router_indices, router_top_value)
        return router_scores, router_indices


class GptOssRMSNorm(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(2880, dtype=torch.bfloat16))

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + 1e-05)
        return (self.weight * hidden_states).to(input_dtype)

def get_mscale(scale, mscale=1):
    return 0.1 * mscale * math.log(scale) + 1.0


def find_correction_dim(num_rotations, dim, base, max_position_embeddings):
    return dim * math.log(max_position_embeddings / (num_rotations * 2 * math.pi)) / (2 * math.log(base))


def find_correction_range(low_rot, high_rot, dim, base, max_position_embeddings):
    low = find_correction_dim(low_rot, dim, base, max_position_embeddings)
    high = find_correction_dim(high_rot, dim, base, max_position_embeddings)
    return (max(low, 0), min(high, dim - 1))


def linear_ramp_factor(min, max, dim):
    linear_func = (torch.arange(dim, dtype=torch.float32) - min) / (max - min)
    ramp_func = torch.clamp(linear_func, 0, 1)
    return ramp_func


class GptOssRotaryEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.attention_scaling = get_mscale(32.0)
        
        low, high = find_correction_range(32.0, 1.0, 64, 150000, 4096)
        inv_freq_extrapolation_factor = 1 - linear_ramp_factor(low, high, 64 // 2)
        pos_freqs = 150000 ** (torch.arange(0, 64, 2).to(dtype=torch.float) / 64)
        inv_freq_extrapolation = 1.0 / pos_freqs
        inv_freq_interpolation = 1.0 / (32.0 * pos_freqs)
        inv_freq = inv_freq_interpolation * (1 - inv_freq_extrapolation_factor) + inv_freq_extrapolation * inv_freq_extrapolation_factor
        
        self.register_buffer('inv_freq', inv_freq, persistent=False)
        self.original_inv_freq = self.inv_freq

    @torch.no_grad()
    def forward(self, x, position_ids):
        inv_freq_expanded = self.inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1).to(x.device)
        position_ids_expanded = position_ids[:, None, :].float()
        device_type = x.device.type if isinstance(x.device.type, str) and x.device.type != 'mps' else 'cpu'
        with torch.autocast(device_type=device_type, enabled=False):
            freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
            emb = freqs
            cos = emb.cos() * self.attention_scaling
            sin = emb.sin() * self.attention_scaling
        return (cos.to(x.dtype), sin.to(x.dtype))


class GptOssExperts(nn.Module):
    def __init__(self):
        super().__init__()
        self.gate_up_proj = nn.Parameter(torch.empty(32, 2880, 2 * 2880, dtype=torch.bfloat16))
        self.gate_up_proj_bias = nn.Parameter(torch.empty(32, 2 * 2880, dtype=torch.bfloat16))
        self.down_proj = nn.Parameter(torch.empty((32, 2880, 2880), dtype=torch.bfloat16))
        self.down_proj_bias = nn.Parameter(torch.empty(32, 2880, dtype=torch.bfloat16))
        self.alpha = 1.702
        self.limit = 7.0

    def forward(self, hidden_states: torch.Tensor, router_indices=None, routing_weights=None) -> torch.Tensor:
        batch_size = hidden_states.shape[0]
        hidden_states = hidden_states.reshape(-1, 2880)
        num_experts = routing_weights.shape[1]

        next_states = torch.zeros_like(hidden_states, dtype=hidden_states.dtype, device=hidden_states.device)
        with torch.no_grad():
            expert_mask = torch.nn.functional.one_hot(router_indices, num_classes=num_experts + 1)
            expert_mask = expert_mask.permute(2, 1, 0)
            expert_hit = torch.greater(expert_mask.sum(dim=(-1, -2)), 0).nonzero()
        for expert_idx in expert_hit[:]:
            expert_idx = expert_idx[0]
            with torch.no_grad():
                _, token_idx = torch.where(expert_mask[expert_idx])
            current_state = hidden_states[token_idx]
            gate_up = current_state @ self.gate_up_proj[expert_idx] + self.gate_up_proj_bias[expert_idx]
            gate, up = (gate_up[..., ::2], gate_up[..., 1::2])
            gate = gate.clamp(min=None, max=self.limit)
            up = up.clamp(min=-self.limit, max=self.limit)
            glu = gate * torch.sigmoid(gate * self.alpha)
            gated_output = (up + 1) * glu
            out = gated_output @ self.down_proj[expert_idx] + self.down_proj_bias[expert_idx]
            weighted_output = out * routing_weights[token_idx, expert_idx, None]
            next_states.index_add_(0, token_idx, weighted_output.to(hidden_states.dtype))
        next_states = next_states.view(batch_size, -1, 2880)

        return next_states


CONFIG_LAYER_TYPES = (
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention',
    'sliding_attention',
    'full_attention'
)


def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)


def eager_attention_forward(
    module: nn.Module,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attention_mask: Optional[torch.Tensor],
    scaling: float,
    dropout: float=0.0,
    # **kwargs
):
    key_states = repeat_kv(key, 8)
    value_states = repeat_kv(value, 8)
    attn_weights = torch.matmul(query, key_states.transpose(2, 3)) * scaling
    if attention_mask is not None:
        causal_mask = attention_mask[:, :, :, :key_states.shape[-2]]
        attn_weights = attn_weights + causal_mask
    sinks = module.sinks.reshape(1, -1, 1, 1).expand(query.shape[0], -1, query.shape[-2], -1)
    combined_logits = torch.cat([attn_weights, sinks], dim=-1)
    combined_logits = combined_logits - combined_logits.max(dim=-1, keepdim=True).values
    probs = F.softmax(combined_logits, dim=-1, dtype=combined_logits.dtype)
    scores = probs[..., :-1]
    attn_weights = nn.functional.dropout(scores, p=dropout, training=False)
    attn_output = torch.matmul(attn_weights, value_states)
    attn_output = attn_output.transpose(1, 2).contiguous()
    return (attn_output, attn_weights)


def _apply_rotary_emb(
    x: torch.Tensor,
    cos: torch.Tensor,
    sin: torch.Tensor,
) -> torch.Tensor:
    first_half, second_half = torch.chunk(x, 2, dim=-1)
    first_ = first_half * cos - second_half * sin
    second_ = second_half * cos + first_half * sin
    return torch.cat((first_, second_), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = _apply_rotary_emb(q, cos, sin)
    k_embed = _apply_rotary_emb(k, cos, sin)
    return q_embed, k_embed


class GptOssAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.q_proj = nn.Linear(2880, 64 * 64, bias=True, dtype=torch.bfloat16)
        self.k_proj = nn.Linear(2880, 8 * 64, bias=True, dtype=torch.bfloat16)
        self.v_proj = nn.Linear(2880, 8 * 64, bias=True, dtype=torch.bfloat16)
        self.o_proj = nn.Linear(64 * 64, 2880, bias=True, dtype=torch.bfloat16)
        self.sinks = nn.Parameter(torch.empty(64, dtype=torch.bfloat16))

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor],
        position_embeddings: Tuple[torch.Tensor, torch.Tensor],
    ) -> tuple[torch.Tensor, torch.Tensor]:
        input_shape = hidden_states.shape[:-1]
        hidden_shape = (*input_shape, -1, 64)
        query_states = self.q_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        key_states = self.k_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        value_states = self.v_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        cos, sin = position_embeddings
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)
        attn_output, attn_weights = eager_attention_forward(
            self,
            query_states,
            key_states,
            value_states,
            attention_mask,
            dropout=0.0,
            scaling=0.125
        )
        attn_output = attn_output.reshape(*input_shape, -1).contiguous()
        attn_output = self.o_proj(attn_output)
        return (attn_output, attn_weights)

# `model.layers.7.mlp.router: GptOssTopKRouter`

In [4]:
inputs_and_outputs = torch.load('model.layers.7.mlp.router.pt')
inputs_and_outputs.keys()

odict_keys(['hidden_states', 'return'])

In [5]:
module = build_module(
    GptOssTopKRouter,
    [(inputs_and_outputs['hidden_states'],)],
    checkpoint_path='model.layers.7.mlp.router.ckpt'
)
module

Neuron: Saving to compiler workdir: /tmp/nxdi_test_d2a10e18-f786-4652-a18d-0e56893226d3/compiler_workdir
Neuron: Using checkpoint path: model.layers.7.mlp.router.ckpt
Neuron: Generating HLOs for the following models: ['GptOssTopKRouter']


[2025-12-09 20:24:16.533: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:24:16.533: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:24:16.533: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:24:16.534: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:24:16.534: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:24:16.534: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:24:16.535: I neuronx_distributed/parallel_layers/parallel_state

Neuron: Generating 1 hlos for key: GptOssTopKRouter
Neuron: Started loading module GptOssTopKRouter
Neuron: Finished loading module GptOssTopKRouter in 0.0012433528900146484 seconds
Neuron: generating HLO: GptOssTopKRouter, input example shape = torch.Size([1, 7, 2880])
Neuron: Finished generating HLO for GptOssTopKRouter in 0.010995626449584961 seconds, input example shape = torch.Size([1, 7, 2880])
Neuron: Generated all HLOs in 0.024591922760009766 seconds
Neuron: Starting compilation for the priority HLO
Neuron: 'GptOssTopKRouter' is the priority model with bucket rank 0


2025-12-09 20:24:16.000566:  4807  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --framework=XLA /tmp/nxdi_test_d2a10e18-f786-4652-a18d-0e56893226d3/compiler_workdir/GptOssTopKRouter/_tp0_bk0/model.MODULE_1c23f89582a2c5788e67+405457e1.hlo_module.pb --output /tmp/nxdi_test_d2a10e18-f786-4652-a18d-0e56893226d3/compiler_workdir/GptOssTopKRouter/_tp0_bk0/model.MODULE_1c23f89582a2c5788e67+405457e1.neff --target=trn1 --enable-saturate-infinity --auto-cast=none --model-type=transformer -O1 --logfile=/tmp/nxdi_test_d2a10e18-f786-4652-a18d-0e56893226d3/compiler_workdir/GptOssTopKRouter/_tp0_bk0/log-neuron-cc.txt --enable-internal-neff-wrapper --verbose=35


/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/libneuronxla/neuron_cc_wrapper.py:283: SyntaxWarning: str format compiler_flags is discouraged as its handling involves repeated joining and splitting, which can easily make mistakes if something is quoted or escaped. Use list[str] instead. Refer to documentation of the Python subprocess module for details.
  warnings.warn(SyntaxWarning(


...

Completed run_backend_driver.

Compiler status PASS
.

Neuron: Done preparing weight layout transformation
Neuron: Sharding Weights for ranks: 0...0


Completed run_backend_driver.

Compiler status PASS
[2025-12-09 20:24:20.920: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:24:20.921: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:24:20.921: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:24:20.921: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:24:20.922: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:24:20.922: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:24:20.922: 

/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/trace/trace.py:642: UserWarning: Removing redundant keys from checkpoint: []
  warnings.warn(f"Removing redundant keys from checkpoint: {keys_to_delete}")
Neuron: Done Sharding weights in 0.012145440000495
Neuron: NxD Model Initialized
Neuron: Finished building model in 9.677208662033081 seconds


NxDModelExecutor(
  original_name=NxDModelExecutor
  (nxd_model): RecursiveScriptModule(
    original_name=NxDModel
    (models): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssTopKRouter): RecursiveScriptModule(original_name=SPMDBucketModelScript)
    )
    (flattener_map): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssTopKRouter): JITWrapper(original_name=JITWrapper)
    )
    (packer): JITWrapper(original_name=JITWrapper)
  )
)

In [6]:
output = module(inputs_and_outputs['hidden_states'])

In [7]:
summary_stats(output, inputs_and_outputs['return'])

Element 0:

Tensor Comparison Results:

Absolute Error (min/median/max):  0/0/0.003906
Relative Error (min/median/max):  0/0/0.007721

Element 1:

Tensor Comparison Results:

Absolute Error (min/median/max):  0/0/0
Relative Error (min/median/max):  0/0/0



# `model.layers.23.post_attention_layernorm: GptOssRMSNorm`

In [8]:
inputs_and_outputs = torch.load('model.layers.23.post_attention_layernorm.pt')
inputs_and_outputs.keys()

odict_keys(['hidden_states', 'return'])

In [9]:
module = build_module(
    GptOssRMSNorm,
    [(inputs_and_outputs['hidden_states'],)],
    checkpoint_path='model.layers.23.post_attention_layernorm.ckpt'
)
module

Neuron: Saving to compiler workdir: /tmp/nxdi_test_494ef06d-8656-4dfd-b396-831267fed7fb/compiler_workdir
Neuron: Using checkpoint path: model.layers.23.post_attention_layernorm.ckpt
Neuron: Generating HLOs for the following models: ['GptOssRMSNorm']


[2025-12-09 20:24:26.240: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:24:26.240: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:24:26.240: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:24:26.241: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:24:26.241: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:24:26.241: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:24:26.242: I neuronx_distributed/parallel_layers/parallel_state

Neuron: Generating 1 hlos for key: GptOssRMSNorm
Neuron: Started loading module GptOssRMSNorm
Neuron: Finished loading module GptOssRMSNorm in 0.0002257823944091797 seconds
Neuron: generating HLO: GptOssRMSNorm, input example shape = torch.Size([1, 7, 2880])
Neuron: Finished generating HLO for GptOssRMSNorm in 0.0027594566345214844 seconds, input example shape = torch.Size([1, 7, 2880])
Neuron: Generated all HLOs in 0.016003847122192383 seconds
Neuron: Starting compilation for the priority HLO
Neuron: 'GptOssRMSNorm' is the priority model with bucket rank 0


2025-12-09 20:24:26.000263:  4807  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --framework=XLA /tmp/nxdi_test_494ef06d-8656-4dfd-b396-831267fed7fb/compiler_workdir/GptOssRMSNorm/_tp0_bk0/model.MODULE_9a5084630e05c9e2eed1+5f3e429c.hlo_module.pb --output /tmp/nxdi_test_494ef06d-8656-4dfd-b396-831267fed7fb/compiler_workdir/GptOssRMSNorm/_tp0_bk0/model.MODULE_9a5084630e05c9e2eed1+5f3e429c.neff --target=trn1 --enable-saturate-infinity --auto-cast=none --model-type=transformer -O1 --logfile=/tmp/nxdi_test_494ef06d-8656-4dfd-b396-831267fed7fb/compiler_workdir/GptOssRMSNorm/_tp0_bk0/log-neuron-cc.txt --enable-internal-neff-wrapper --verbose=35


/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/libneuronxla/neuron_cc_wrapper.py:283: SyntaxWarning: str format compiler_flags is discouraged as its handling involves repeated joining and splitting, which can easily make mistakes if something is quoted or escaped. Use list[str] instead. Refer to documentation of the Python subprocess module for details.
  warnings.warn(SyntaxWarning(


.

Neuron: Done compilation for the priority HLO in 2.2351222038269043 seconds
Neuron: No changes on weight layout, skip updating weight layout for other HLOs
Neuron: Starting compilation for all HLOs
Neuron: Finished Compilation for all HLOs in 0.000377655029296875 seconds
Neuron: No changes on weight layout, falling back to the existing weight layout
Neuron: Sharding Weights for ranks: 0...0


Completed run_backend_driver.

Compiler status PASS
[2025-12-09 20:24:28.535: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:24:28.535: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:24:28.536: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:24:28.536: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:24:28.537: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:24:28.537: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:24:28.538: 

/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/trace/trace.py:642: UserWarning: Removing redundant keys from checkpoint: []
  warnings.warn(f"Removing redundant keys from checkpoint: {keys_to_delete}")
Neuron: Done Sharding weights in 0.012529089999588905
Neuron: NxD Model Initialized
Neuron: Finished building model in 2.320403814315796 seconds


NxDModelExecutor(
  original_name=NxDModelExecutor
  (nxd_model): RecursiveScriptModule(
    original_name=NxDModel
    (models): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssRMSNorm): RecursiveScriptModule(original_name=SPMDBucketModelScript)
    )
    (flattener_map): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssRMSNorm): JITWrapper(original_name=JITWrapper)
    )
    (packer): JITWrapper(original_name=JITWrapper)
  )
)

In [10]:
output = module(inputs_and_outputs['hidden_states'])

In [11]:
summary_stats(output, inputs_and_outputs['return'])

Tensor Comparison Results:

Absolute Error (min/median/max):  0/0/0
Relative Error (min/median/max):  0/0/0


# `model.rotary_emb: GptOssRotaryEmbedding`

In [12]:
inputs_and_outputs = torch.load('model.rotary_emb.pt')
inputs_and_outputs.keys()

odict_keys(['x', 'position_ids', 'return'])

In [13]:
module = build_module(
    GptOssRotaryEmbedding,
    [(inputs_and_outputs['x'], inputs_and_outputs['position_ids'])],
)
module

Neuron: Saving to compiler workdir: /tmp/nxdi_test_7f82e13d-aa4e-4caa-b4bf-53c5e8ebc22d/compiler_workdir
Neuron: Using checkpoint path: /tmp/nxdi_test_7f82e13d-aa4e-4caa-b4bf-53c5e8ebc22d/checkpoint.pt


[2025-12-09 20:24:28.589: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:24:28.589: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:24:28.589: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:24:28.590: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:24:28.590: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:24:28.620: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:24:28.620: I neuronx_distributed/parallel_layers/parallel_state

Neuron: Generating HLOs for the following models: ['GptOssRotaryEmbedding']


[2025-12-09 20:24:28.626: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:24:28.627: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:24:28.627: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:24:28.627: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:24:28.628: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:24:28.628: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:24:28.629: I neuronx_distributed/parallel_layers/parallel_state

Neuron: Generating 1 hlos for key: GptOssRotaryEmbedding
Neuron: Started loading module GptOssRotaryEmbedding
Neuron: Finished loading module GptOssRotaryEmbedding in 0.00039076805114746094 seconds
Neuron: generating HLO: GptOssRotaryEmbedding, input example shape = torch.Size([1, 7, 2880])
/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:289: UserWarning: Received an input tensor that was unused or used in a non-static way when traced so the tensor will be ignored. (index=0, shape=torch.Size([1, 7, 2880]), dtype=torch.bfloat16). The non-static usage could happen when the traced function expects the input tensor's shape to change (i.e., using the shape to do index slicing), which is not allowed by inference trace expecting static input shapes.
  warnings.warn(
Neuron: Finished generating HLO for GptOssRotaryEmbedding in 0.0029935836791992188 seconds, input example shape = torch.Size([1, 7, 2880])
Neuron: Generated all

...

.

Neuron: Done compilation for the priority HLO in 2.092477798461914 seconds
Neuron: No changes on weight layout, skip updating weight layout for other HLOs
Neuron: Starting compilation for all HLOs
Neuron: Finished Compilation for all HLOs in 0.0003654956817626953 seconds
Neuron: No changes on weight layout, falling back to the existing weight layout
Neuron: Sharding Weights for ranks: 0...0


Completed run_backend_driver.

Compiler status PASS
[2025-12-09 20:24:30.781: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:24:30.782: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:24:30.782: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:24:30.783: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:24:30.783: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:24:30.783: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:24:30.784: 

/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/trace/trace.py:642: UserWarning: Removing redundant keys from checkpoint: []
  warnings.warn(f"Removing redundant keys from checkpoint: {keys_to_delete}")
Neuron: Done Sharding weights in 0.012313436000113143
Neuron: NxD Model Initialized
Neuron: Finished building model in 2.1781702041625977 seconds


NxDModelExecutor(
  original_name=NxDModelExecutor
  (nxd_model): RecursiveScriptModule(
    original_name=NxDModel
    (models): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssRotaryEmbedding): RecursiveScriptModule(original_name=SPMDBucketModelScript)
    )
    (flattener_map): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssRotaryEmbedding): JITWrapper(original_name=JITWrapper)
    )
    (packer): JITWrapper(original_name=JITWrapper)
  )
)

In [14]:
output = module(inputs_and_outputs['x'], inputs_and_outputs['position_ids'])

In [15]:
summary_stats(output, inputs_and_outputs['return'])

Element 0:

Tensor Comparison Results:

Absolute Error (min/median/max):  0/0/0
Relative Error (min/median/max):  0/0/0

Element 1:

Tensor Comparison Results:

Absolute Error (min/median/max):  0/0/0
Relative Error (min/median/max):  0/0/0



# `model.layers.4.mlp.experts: GptOssExperts`

In [16]:
inputs_and_outputs = torch.load('model.layers.4.mlp.experts.pt')
inputs_and_outputs.keys()

odict_keys(['hidden_states', 'router_indices', 'routing_weights', 'return'])

In [17]:
module = build_module(
    GptOssExperts,
    [(inputs_and_outputs['hidden_states'], inputs_and_outputs['router_indices'], inputs_and_outputs['routing_weights'])],
    checkpoint_path='model.layers.4.mlp.experts.ckpt'
)
module

Neuron: Saving to compiler workdir: /tmp/nxdi_test_8ded87a6-8cad-4441-b6c0-31d291841060/compiler_workdir
Neuron: Using checkpoint path: model.layers.4.mlp.experts.ckpt
Neuron: Generating HLOs for the following models: ['GptOssExperts']


[2025-12-09 20:24:30.836: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:24:30.836: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:24:30.836: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:24:30.837: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:24:30.837: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:24:30.838: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:24:30.838: I neuronx_distributed/parallel_layers/parallel_state

Neuron: Generating 1 hlos for key: GptOssExperts
Neuron: Started loading module GptOssExperts
Neuron: Finished loading module GptOssExperts in 0.0003979206085205078 seconds
Neuron: generating HLO: GptOssExperts, input example shape = torch.Size([1, 7, 2880])
/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:289: UserWarning: Received an input tensor that was unused or used in a non-static way when traced so the tensor will be ignored. (index=1, shape=torch.Size([7, 4]), dtype=torch.int64). The non-static usage could happen when the traced function expects the input tensor's shape to change (i.e., using the shape to do index slicing), which is not allowed by inference trace expecting static input shapes.
  warnings.warn(
Neuron: Finished generating HLO for GptOssExperts in 2.261340856552124 seconds, input example shape = torch.Size([1, 7, 2880])
Neuron: Generated all HLOs in 2.2778210639953613 seconds
Neuron: Starting c

2025-12-09 20:24:33.000144:  4807  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --framework=XLA /tmp/nxdi_test_8ded87a6-8cad-4441-b6c0-31d291841060/compiler_workdir/GptOssExperts/_tp0_bk0/model.MODULE_29366dfb042adeb577cd+cf4bb07b.hlo_module.pb --output /tmp/nxdi_test_8ded87a6-8cad-4441-b6c0-31d291841060/compiler_workdir/GptOssExperts/_tp0_bk0/model.MODULE_29366dfb042adeb577cd+cf4bb07b.neff --target=trn1 --enable-saturate-infinity --auto-cast=none --model-type=transformer -O1 --logfile=/tmp/nxdi_test_8ded87a6-8cad-4441-b6c0-31d291841060/compiler_workdir/GptOssExperts/_tp0_bk0/log-neuron-cc.txt --enable-internal-neff-wrapper --verbose=35


/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/libneuronxla/neuron_cc_wrapper.py:283: SyntaxWarning: str format compiler_flags is discouraged as its handling involves repeated joining and splitting, which can easily make mistakes if something is quoted or escaped. Use list[str] instead. Refer to documentation of the Python subprocess module for details.
  warnings.warn(SyntaxWarning(


...

Completed run_backend_driver.

Compiler status PASS
.

Neuron: Done preparing weight layout transformation
Neuron: Sharding Weights for ranks: 0...0


Completed run_backend_driver.

Compiler status PASS
[2025-12-09 20:25:01.924: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:25:01.925: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:25:01.925: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:25:01.925: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:25:01.926: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:25:01.926: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:25:01.926: 

/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/trace/trace.py:642: UserWarning: Removing redundant keys from checkpoint: []
  warnings.warn(f"Removing redundant keys from checkpoint: {keys_to_delete}")
Neuron: Done Sharding weights in 2.1754744519994347
Neuron: NxD Model Initialized
Neuron: Finished building model in 33.706507205963135 seconds


NxDModelExecutor(
  original_name=NxDModelExecutor
  (nxd_model): RecursiveScriptModule(
    original_name=NxDModel
    (models): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssExperts): RecursiveScriptModule(original_name=SPMDBucketModelScript)
    )
    (flattener_map): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssExperts): JITWrapper(original_name=JITWrapper)
    )
    (packer): JITWrapper(original_name=JITWrapper)
  )
)

In [18]:
output = module(inputs_and_outputs['hidden_states'], inputs_and_outputs['router_indices'], inputs_and_outputs['routing_weights'])

In [19]:
summary_stats(output, inputs_and_outputs['return'])

Tensor Comparison Results:

Absolute Error (min/median/max):  0/0.003906/1
Relative Error (min/median/max):  0/0.004059/1


In [20]:
output

tensor([[[-0.0918,  0.3867,  0.2090,  ...,  0.3652, -0.3984, -0.2188],
         [-0.1357, -0.1484,  0.1797,  ..., -0.2695,  0.0559, -0.2139],
         [ 0.6797, -0.0820,  0.5430,  ..., -0.1006, -0.3555, -0.5117],
         ...,
         [-0.1074, -0.6250, -0.0918,  ...,  0.2891,  0.1699,  0.2168],
         [ 0.6562, -0.7969,  0.8203,  ..., -0.9609, -0.3008,  0.2168],
         [-0.4805, -2.3750,  0.5078,  ..., -0.0166,  0.5234,  0.2637]]],
       dtype=torch.bfloat16)

In [21]:
inputs_and_outputs['return']

tensor([[[-0.0918,  0.3906,  0.2100,  ...,  0.3633, -0.3984, -0.2207],
         [-0.1289, -0.1621,  0.1836,  ..., -0.2676,  0.0559, -0.2188],
         [ 0.6719, -0.0781,  0.5547,  ..., -0.1011, -0.3535, -0.5078],
         ...,
         [-0.0996, -0.6094, -0.0850,  ...,  0.2988,  0.1650,  0.2148],
         [ 0.6562, -0.7852,  0.8203,  ..., -0.9609, -0.2988,  0.2158],
         [-0.4844, -2.3594,  0.4941,  ..., -0.0195,  0.5312,  0.2656]]],
       dtype=torch.bfloat16)

# `model.layers.8.self_attn: GptOssAttention`

In [22]:
inputs_and_outputs = torch.load('model.layers.8.self_attn.pt')
inputs_and_outputs.keys()

odict_keys(['hidden_states', 'attention_mask', 'position_embeddings', 'return'])

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[24], line 1
----> 1 module = build_module(
      2     GptOssAttention,
      3     [(inputs_and_outputs['hidden_states'], inputs_and_outputs['attention_mask'], inputs_and_outputs['position_embeddings'])],
      4     checkpoint_path='model.layers.8.self_attn.ckpt'
      5 )
      6 module

File /opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed_inference/utils/testing.py:199, in build_module(module_cls, example_inputs, module_init_kwargs, tp_degree, compiler_args, compiler_workdir, checkpoint_path, priority_model_idx, logical_nc_config, dry_run, checkpoint_loader_fn)
    173 """
    174 Compiles a module to Neuron.
    175 
   (...)
    196     The Neuron model, or None if dry run mode is enabled.
    197 """
    198 if not _is_tensor_tuple_list(example_inputs):
--> 199     raise ValueError("example_inputs must be a list of tensor tuples")
    200 if len(example_inputs) != 1:
    201     # Bucketing isn't currently supported for this utility.
    202     raise ValueError("example_inputs must contain exactly one input")

ValueError: example_inputs must be a list of tensor tuples
```

In [23]:
class GptOssAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.q_proj = nn.Linear(2880, 64 * 64, bias=True, dtype=torch.bfloat16)
        self.k_proj = nn.Linear(2880, 8 * 64, bias=True, dtype=torch.bfloat16)
        self.v_proj = nn.Linear(2880, 8 * 64, bias=True, dtype=torch.bfloat16)
        self.o_proj = nn.Linear(64 * 64, 2880, bias=True, dtype=torch.bfloat16)
        self.sinks = nn.Parameter(torch.empty(64, dtype=torch.bfloat16))

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor],
        cos: torch.Tensor,
        sin: torch.Tensor,
    ) -> tuple[torch.Tensor, torch.Tensor]:
        input_shape = hidden_states.shape[:-1]
        hidden_shape = (*input_shape, -1, 64)
        query_states = self.q_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        key_states = self.k_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        value_states = self.v_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)
        attn_output, attn_weights = eager_attention_forward(
            self,
            query_states,
            key_states,
            value_states,
            attention_mask,
            dropout=0.0,
            scaling=0.125
        )
        attn_output = attn_output.reshape(*input_shape, -1).contiguous()
        attn_output = self.o_proj(attn_output)
        return (attn_output, attn_weights)

In [24]:
module = build_module(
    GptOssAttention,
    [(inputs_and_outputs['hidden_states'], inputs_and_outputs['attention_mask'], inputs_and_outputs['position_embeddings'][0], inputs_and_outputs['position_embeddings'][1])],
    checkpoint_path='model.layers.8.self_attn.ckpt'
)
module

Neuron: Saving to compiler workdir: /tmp/nxdi_test_b1a04fe3-81d7-4865-925a-b2937c0679f2/compiler_workdir
Neuron: Using checkpoint path: model.layers.8.self_attn.ckpt
Neuron: Generating HLOs for the following models: ['GptOssAttention']


[2025-12-09 20:25:04.637: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:25:04.637: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:25:04.638: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:25:04.638: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:25:04.638: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:25:04.639: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:25:04.639: I neuronx_distributed/parallel_layers/parallel_state

Neuron: Generating 1 hlos for key: GptOssAttention
Neuron: Started loading module GptOssAttention
Neuron: Finished loading module GptOssAttention in 0.1647791862487793 seconds
Neuron: generating HLO: GptOssAttention, input example shape = torch.Size([1, 7, 2880])
Neuron: Finished generating HLO for GptOssAttention in 0.0631873607635498 seconds, input example shape = torch.Size([1, 7, 2880])
Neuron: Generated all HLOs in 0.24099278450012207 seconds
Neuron: Starting compilation for the priority HLO
Neuron: 'GptOssAttention' is the priority model with bucket rank 0


2025-12-09 20:25:04.000891:  4807  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --framework=XLA /tmp/nxdi_test_b1a04fe3-81d7-4865-925a-b2937c0679f2/compiler_workdir/GptOssAttention/_tp0_bk0/model.MODULE_68acbf1f8e142092eeee+3c85e442.hlo_module.pb --output /tmp/nxdi_test_b1a04fe3-81d7-4865-925a-b2937c0679f2/compiler_workdir/GptOssAttention/_tp0_bk0/model.MODULE_68acbf1f8e142092eeee+3c85e442.neff --target=trn1 --enable-saturate-infinity --auto-cast=none --model-type=transformer -O1 --logfile=/tmp/nxdi_test_b1a04fe3-81d7-4865-925a-b2937c0679f2/compiler_workdir/GptOssAttention/_tp0_bk0/log-neuron-cc.txt --enable-internal-neff-wrapper --verbose=35


/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/libneuronxla/neuron_cc_wrapper.py:283: SyntaxWarning: str format compiler_flags is discouraged as its handling involves repeated joining and splitting, which can easily make mistakes if something is quoted or escaped. Use list[str] instead. Refer to documentation of the Python subprocess module for details.
  warnings.warn(SyntaxWarning(


...

Completed run_backend_driver.

Compiler status PASS
.

Neuron: Done preparing weight layout transformation
Neuron: Sharding Weights for ranks: 0...0


Completed run_backend_driver.

Compiler status PASS
[2025-12-09 20:25:12.171: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-09 20:25:12.171: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-09 20:25:12.171: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-09 20:25:12.172: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-09 20:25:12.172: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-09 20:25:12.172: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x741ea2eb7c70>, 'Ascending Ring PG Group')>
[2025-12-09 20:25:12.173: 

/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/trace/trace.py:642: UserWarning: Removing redundant keys from checkpoint: []
  warnings.warn(f"Removing redundant keys from checkpoint: {keys_to_delete}")
Neuron: Done Sharding weights in 0.04947821399946406
Neuron: NxD Model Initialized
Neuron: Finished building model in 7.660136938095093 seconds


NxDModelExecutor(
  original_name=NxDModelExecutor
  (nxd_model): RecursiveScriptModule(
    original_name=NxDModel
    (models): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssAttention): RecursiveScriptModule(original_name=SPMDBucketModelScript)
    )
    (flattener_map): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssAttention): JITWrapper(original_name=JITWrapper)
    )
    (packer): JITWrapper(original_name=JITWrapper)
  )
)

In [25]:
output = module(inputs_and_outputs['hidden_states'], inputs_and_outputs['attention_mask'], inputs_and_outputs['position_embeddings'][0], inputs_and_outputs['position_embeddings'][1])

In [26]:
summary_stats(output, inputs_and_outputs['return'])

Element 0:

Tensor Comparison Results:

Absolute Error (min/median/max):  0/0.01562/2
Relative Error (min/median/max):  0/0.007568/1

Element 1:

Tensor Comparison Results:

Absolute Error (min/median/max):  0/0/0.02344
Relative Error (min/median/max):  0/0/0.08447



In [27]:
output[0]

tensor([[[ 2.0000,  6.9375, -1.8281,  ...,  3.0625, -1.3438,  0.4102],
         [ 0.2148, -1.4297,  0.7617,  ..., -0.5234, -0.0154, -0.2441],
         [-1.6797, -0.8516,  0.3418,  ..., -0.6172, -0.0894, -0.2021],
         ...,
         [-2.1094, -0.4727,  0.2812,  ..., -1.0625,  0.7578, -0.3496],
         [ 0.3203, -2.2969,  0.7812,  ..., -0.8828,  0.1011,  0.1953],
         [-2.1875,  0.8750,  2.2969,  ..., -2.3125,  0.0535, -0.6094]]],
       dtype=torch.bfloat16)

In [28]:
inputs_and_outputs['return'][0]

tensor([[[ 1.9922,  6.9062, -1.8359,  ...,  3.0469, -1.3281,  0.4219],
         [ 0.2090, -1.4297,  0.7461,  ..., -0.5000, -0.0143, -0.2383],
         [-1.7031, -0.8594,  0.3535,  ..., -0.6016, -0.0942, -0.1865],
         ...,
         [-2.1094, -0.5078,  0.2754,  ..., -1.0312,  0.7578, -0.3281],
         [ 0.3340, -2.2812,  0.7734,  ..., -0.8828,  0.1045,  0.1846],
         [-2.1562,  0.9102,  2.2812,  ..., -2.2969,  0.0447, -0.6289]]],
       dtype=torch.bfloat16)